### The Data

In [1]:
import pandas as pd

In [2]:
# https://www.kaggle.com/nikhilmittal/flight-fare-prediction-mh
df = pd.read_excel('data/train.xlsx')

In [3]:
df.sample(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
10203,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,11:30,04:25 07 Jun,16h 55m,1 stop,In-flight meal not included,10262
3442,Multiple carriers,1/06/2019,Delhi,Cochin,DEL → BOM → COK,12:50,01:30 02 Jun,12h 40m,1 stop,No info,6795
3775,Jet Airways,12/05/2019,Kolkata,Banglore,CCU → BOM → BLR,06:30,09:20 13 May,26h 50m,1 stop,In-flight meal not included,9663
1620,Jet Airways,3/05/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
5307,IndiGo,3/06/2019,Delhi,Cochin,DEL → HYD → COK,06:50,16:10,9h 20m,1 stop,No info,7514


In [4]:
df.shape

(10683, 11)

### EDA

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Price,10683.0,9087.064121,4611.359167,1759.0,5277.0,8372.0,12373.0,79512.0


In [6]:
f = ['Airline', 'Source', 'Destination', 'Route', 'Additional_Info']

for fi in f:
    print(f'{fi}:\n')
    print(df[fi].value_counts())
    print('\n')

Airline:

Jet Airways                          3849
IndiGo                               2053
Air India                            1752
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: Airline, dtype: int64


Source:

Delhi       4537
Kolkata     2871
Banglore    2197
Mumbai       697
Chennai      381
Name: Source, dtype: int64


Destination:

Cochin       4537
Banglore     2871
Delhi        1265
New Delhi     932
Hyderabad     697
Kolkata       381
Name: Destination, dtype: int64


Route:

DEL → BOM → COK                2376
BLR → DEL                      1552
CCU → BOM → BLR                 979
CCU → BLR                       724
BOM → HYD                

### The Cleaning

In [7]:
# lower case everything
df.columns = [c.lower() for c in df.columns]
# date to date
df['date_of_journey'] = df['date_of_journey'].apply(pd.to_datetime)
# price to USD
df['price'] = df['price'].apply(lambda x: round(x * 0.014))
# stops to number
df['total_stops'] = df['total_stops'].apply(
    lambda x: pd.to_numeric(str(x).split(' ')[0], errors='coerce')
)
df['total_stops'] = df['total_stops'].fillna(0)
# rename columns 
df = df.rename(columns={
    'date_of_journey': 'date', 
    'total_stops': 'stops',
    'source': 'origin'
})

In [8]:
df.head()

,airline,date,origin,destination,route,dep_time,arrival_time,duration,stops,additional_info,price
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,0.0,No info,55
1,Air India,2019-01-05,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2.0,No info,107
2,Jet Airways,2019-09-06,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2.0,No info,194
3,IndiGo,2019-12-05,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1.0,No info,87
4,IndiGo,2019-01-03,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1.0,No info,186


In [9]:
df[['price']].describe().T

,count,mean,std,min,25%,50%,75%,max
price,10683.0,127.208462,64.56394,25.0,74.0,117.0,173.0,1113.0


### Select + Split

In [10]:
y = df['price']
X = df[['date', 'origin', 'destination', 'stops']]

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
X_train

,date,origin,destination,stops
752,2019-05-27,Delhi,Cochin,1.0
7431,2019-06-05,Chennai,Kolkata,0.0
5478,2019-12-06,Kolkata,Banglore,1.0
3445,2019-06-06,Delhi,Cochin,1.0
9380,2019-06-18,Banglore,Delhi,0.0
...,...,...,...,...
5734,2019-03-27,Delhi,Cochin,1.0
5191,2019-09-05,Kolkata,Banglore,1.0
5390,2019-05-15,Delhi,Cochin,1.0
860,2019-03-03,Banglore,New Delhi,0.0


### Dealing with Dates

In [14]:
X_train['date'].head()

752    2019-05-27
7431   2019-06-05
5478   2019-12-06
3445   2019-06-06
9380   2019-06-18
Name: date, dtype: datetime64[ns]

In [15]:
X_train['date'].dt.month.head()

752      5
7431     6
5478    12
3445     6
9380     6
Name: date, dtype: int64

In [16]:
X_train['date'].dt.dayofweek.head()

752     0
7431    2
5478    4
3445    3
9380    1
Name: date, dtype: int64

In [17]:
class DateEncoder:
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        month = X.dt.month
        day_of_week = X.dt.dayofweek
        return pd.concat([month, day_of_week], axis=1)
    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)

In [18]:
de = DateEncoder()

de.fit_transform(X_train['date']).head()

,date,date
752,5,0
7431,6,2
5478,12,4
3445,6,3
9380,6,1


In [19]:
de.transform(X_test['date'])

,date,date
6076,5,5
3544,3,2
7313,1,5
5032,5,4
2483,5,1
...,...,...
3022,3,0
8417,5,2
2605,6,4
1392,6,3


In [20]:
from sklearn.preprocessing import LabelBinarizer
from sklearn_pandas import DataFrameMapper

In [21]:
X_train.head(5)

,date,origin,destination,stops
752,2019-05-27,Delhi,Cochin,1.0
7431,2019-06-05,Chennai,Kolkata,0.0
5478,2019-12-06,Kolkata,Banglore,1.0
3445,2019-06-06,Delhi,Cochin,1.0
9380,2019-06-18,Banglore,Delhi,0.0


In [22]:
mapper = DataFrameMapper([
    ('date', DateEncoder(), {'input_df': True}),
    ('origin', LabelBinarizer()), 
    ('destination', LabelBinarizer()),
    ('stops', None)
], df_out=True)

In [23]:
Z_train = mapper.fit_transform(X_train)

In [25]:
Z_train

,date_0,date_1,origin_Banglore,origin_Chennai,origin_Delhi,origin_Kolkata,origin_Mumbai,destination_Banglore,destination_Cochin,destination_Delhi,destination_Hyderabad,destination_Kolkata,destination_New Delhi,stops
752,5,0,0,0,1,0,0,0,1,0,0,0,0,1.0
7431,6,2,0,1,0,0,0,0,0,0,0,1,0,0.0
5478,12,4,0,0,0,1,0,1,0,0,0,0,0,1.0
3445,6,3,0,0,1,0,0,0,1,0,0,0,0,1.0
9380,6,1,1,0,0,0,0,0,0,1,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,3,2,0,0,1,0,0,0,1,0,0,0,0,1.0
5191,9,3,0,0,0,1,0,1,0,0,0,0,0,1.0
5390,5,2,0,0,1,0,0,0,1,0,0,0,0,1.0
860,3,6,1,0,0,0,0,0,0,0,0,0,1,0.0


In [24]:
Z_test = mapper.transform(X_test)

### The Model

In [26]:
from sklearn.linear_model import LinearRegression

In [27]:
model = LinearRegression()

In [28]:
model.fit(Z_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [29]:
model.score(Z_train, y_train)

0.4086963860270111

In [30]:
model.score(Z_test, y_test)

0.419190048886199

In [31]:
from sklearn.metrics import mean_squared_error

In [32]:
mean_squared_error(y_test, model.predict(Z_test))**(1/2)

48.22128370211856

In [33]:
from sklearn.dummy import DummyRegressor

dummy = DummyRegressor()
dummy.fit(Z_train, y_train)
print(dummy.score(Z_train, y_train))
print(dummy.score(Z_test, y_test))

mean_squared_error(y_test, dummy.predict(Z_test))**(1/2)

0.0
-3.1184373658454945e-05


63.27446980630515

### Pipeline

In [34]:
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(mapper, model)

In [35]:
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.419190048886199

In [42]:
X_train.sample().to_dict(orient='list')

{'date': [Timestamp('2019-06-05 00:00:00')],
 'origin': ['Chennai'],
 'destination': ['Kolkata'],
 'stops': [0.0]}

In [44]:
new = pd.DataFrame({
    'date': [pd.Timestamp('2020-03-09 00:00:00')],
    'origin': ['Banglore'],
    'destination': ['Kolkata'],
    'stops': [0]
})

In [45]:
pipe.predict(new)

array([86.60683116])

#### Pickle 

In [46]:
import pickle

In [47]:
with open('pipe.pkl', 'wb') as f:
    pickle.dump(pipe, f)

del pipe

In [48]:
with open('pipe.pkl', 'rb') as f:
    pipe = pickle.load(f)

In [49]:
pipe.predict(new)[0]

86.60683116420851